<a href="https://colab.research.google.com/github/GaryPython/30-day-for-python/blob/main/Titanic/7_%E7%89%B9%E5%BE%B5%E9%81%B8%E5%8F%96_%E4%BD%9C%E6%A5%ADD40.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 今天的作業
在鐵達尼資料集中，以Titanic_train.csv 中，首先將有遺失值的數值刪除，試著將課堂中所學的方法應用上去。
* Q1: 目標變數為 Survived，以 Pclass, Age, SibSp, Parch, Fare,Sex, Embarked 為特徵，試著用今天教授的包裝法，搭配課程所教的 SVC，試著排出其餘特徵的重要性。


In [1]:
# library
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import math
import statistics
import seaborn as sns
from IPython.display import display
import sklearn
print(sklearn.__version__)
#如果只有 0.19 記得要更新至 最新版本
print(pd.__version__)
%matplotlib inline




# 特徵選取會用到的函數
from sklearn.feature_selection import VarianceThreshold
from sklearn import preprocessing

from sklearn.datasets import make_friedman1
from sklearn.feature_selection import RFE
from sklearn.svm import SVC

 

0.22.2.post1
1.1.5


## 讀入資料

In [2]:
from google.colab import files
uploaded = files.upload()

Saving Titanic_train.csv to Titanic_train.csv


In [72]:
df_train = pd.read_csv("Titanic_train.csv")
print(df_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


In [73]:
#1.產稱一個新的變數 Survived_cate ，資料型態傳換成類別型態      
#2.把題目中的 Survived 用 Survived_cate 來做分析    
df_train['Survived_cate']=df_train['Survived']

df_train['Survived_cate']=df_train['Survived_cate'].astype('object')
print(df_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   PassengerId    891 non-null    int64  
 1   Survived       891 non-null    int64  
 2   Pclass         891 non-null    int64  
 3   Name           891 non-null    object 
 4   Sex            891 non-null    object 
 5   Age            714 non-null    float64
 6   SibSp          891 non-null    int64  
 7   Parch          891 non-null    int64  
 8   Ticket         891 non-null    object 
 9   Fare           891 non-null    float64
 10  Cabin          204 non-null    object 
 11  Embarked       889 non-null    object 
 12  Survived_cate  891 non-null    object 
dtypes: float64(2), int64(5), object(6)
memory usage: 90.6+ KB
None


In [74]:
df_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived_cate
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,0
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,1
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,0
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,1


In [75]:
#我們先把遺失值刪除
## 取出資料後，把遺失值刪除
complete_data=df_train.dropna()

#排除 Ｎame 當特徵，先刪除每一個人都是獨特的資料，先檢視其他變數
complete_data=complete_data.drop(['Name','Ticket','PassengerId'], axis=1)

display(complete_data.head(5))
print(complete_data.shape)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Survived_cate
1,1,1,female,38.0,1,0,71.2833,C85,C,1
3,1,1,female,35.0,1,0,53.1000,C123,S,1
6,0,1,male,54.0,0,0,51.8625,E46,S,0
10,1,3,female,4.0,1,1,16.7000,G6,S,1
11,1,1,female,58.0,0,0,26.5500,C103,S,1


(183, 10)


In [76]:
num_features = []
for dtype, feature in zip(complete_data.dtypes, complete_data.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
       
print(f'{len(num_features)} Numeric Features : {num_features}\n')

6 Numeric Features : ['Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare']



In [77]:

cat_features = []
for dtype, feature in zip(complete_data.dtypes, complete_data.columns):
    if dtype == 'object':
        cat_features.append(feature)
print(f'{len(cat_features)} category Features : {cat_features}\n')

4 category Features : ['Sex', 'Cabin', 'Embarked', 'Survived_cate']



## 定義 x,y

In [78]:
x=complete_data[['Pclass', 'Age', 'SibSp', 'Parch', 'Fare','Sex', 'Embarked']]
y=complete_data['Survived']
display(x.head(5))
display(y.head(5))

,Pclass,Age,SibSp,Parch,Fare,Sex,Embarked
1,1,38.0,1,0,71.2833,female,C
3,1,35.0,1,0,53.1000,female,S
6,1,54.0,0,0,51.8625,male,S
10,3,4.0,1,1,16.7000,female,S
11,1,58.0,0,0,26.5500,female,S


1     1
3     1
6     0
10    1
11    1
Name: Survived, dtype: int64

### Q1: 目標變數為 Survived，試著用今天教授的包裝法，搭配課程所教的 SVC，試著排出其餘特徵的重要性!


[cat.code&cat.categories](https://bbs.csdn.net/topics/394349885)

In [79]:
#離散要轉換成數值，['Sex', 'Embarked']

x = x.copy()
#print(x)
c = x.loc[:,'Sex'].astype('category')#節省記憶體
#print(c)
print(c.cat.categories)
d = dict(enumerate(c.cat.categories))
print(d)
x.loc[:,'Sex']=x.loc[:,'Sex'].astype('category').cat.codes # ok
display(x.head(5))

Index(['female', 'male'], dtype='object')
{0: 'female', 1: 'male'}


,Pclass,Age,SibSp,Parch,Fare,Sex,Embarked
1,1,38.0,1,0,71.2833,0,C
3,1,35.0,1,0,53.1000,0,S
6,1,54.0,0,0,51.8625,1,S
10,3,4.0,1,1,16.7000,0,S
11,1,58.0,0,0,26.5500,0,S


In [80]:
c = x.loc[:,'Embarked'].astype('category')
d = dict(enumerate(c.cat.categories))
print (d)
x.loc[:,'Embarked']=x.loc[:,'Embarked'].astype('category').cat.codes # ok
display(x.head(5))


{0: 'C', 1: 'Q', 2: 'S'}


,Pclass,Age,SibSp,Parch,Fare,Sex,Embarked
1,1,38.0,1,0,71.2833,0,0
3,1,35.0,1,0,53.1000,0,2
6,1,54.0,0,0,51.8625,1,2
10,3,4.0,1,1,16.7000,0,2
11,1,58.0,0,0,26.5500,0,2


In [92]:
#執行 包裝法

from sklearn.datasets import make_friedman1
from sklearn.feature_selection import RFE
from sklearn.svm import SVC
estimator = SVC(kernel="linear")
selector = RFE(estimator, n_features_to_select=1, step=1)
#n_features_to_select：最後要選擇留下多少特徵。
#Step：刪除法，每一部刪除多少特徵。
selector = selector.fit(x, y)
## True= selected feature
print(selector.support_)
#Selected (i.e., estimated best) features are assigned rank 1.
ranking=selector.ranking_
print(ranking)
print(x.loc[:,selector.support_].columns)
rfe_feature = x.loc[:,selector.support_].columns.tolist()
print(rfe_feature)
#Pclass-Age-SibSp-Parch-Fare-Sex-Embarked 的依序排序為 [3 5 2 6 7 1 4]

[False False False False False  True False]
[3 5 4 6 7 1 2]
Index(['Sex'], dtype='object')
['Sex']


[列表,陣列,矩陣兩兩轉換tolist()](https://codertw.com/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80/358261/)